In [134]:
import pandas as pd
import nltk
import numpy as np
from functools import reduce
import math


In [133]:
NOME_DO_ARQUIVO = 'estadao_noticias_eleicao.csv'
indice_invertido = {}
dados = pd.read_csv(NOME_DO_ARQUIVO)
dados = dados.replace(np.nan, '', regex=True)
dados["TF"] = None

In [135]:
def geraIndice():
    for i in range(dados.titulo.size):
        texto = dados.titulo[i] + " " + dados.conteudo[i] + " " + dados.subTitulo[i]
        palavras_divididas = nltk.word_tokenize(texto)
        dados.TF[i] = {}
        for palavra in palavras_divididas:
            palavra = palavra.lower()
            if palavra not in indice_invertido:
                indice_invertido[palavra] = set([dados.idNoticia[i]])
            else:
                indice_invertido[palavra].add(dados.idNoticia[i])
            if palavra not in dados.TF[i]:
                dados.TF[i][palavra] = 0
            dados.TF[i][palavra] += 1

In [83]:
def modeloBinario(termos):
    return list(reduce((lambda x,y: indice_invertido[x] & indice_invertido[y]), termos))

In [89]:
def getDocumentsWithTerms(terms):
    return list(reduce((lambda x,y: indice_invertido[x].union(indice_invertido[y])), terms))

In [85]:
def generateQueryTF(query):
    tf = {}
    for term in query:
        if term not in tf:
            tf[term] = 0
        tf[term] += 1
    return tf

In [144]:
def modeloTF(termos):
    documentos = getDocumentsWithTerms(termos)
    tfQuery = generateQueryTF(termos)
    documentsTF = []
    for idDoc in documentos:
        documentWeight = 0
        for queryTerm in tfQuery:
            docTF = dados.TF[idDoc][queryTerm] if queryTerm in dados.TF[idDoc] else 0
            qTF = tfQuery[queryTerm] if queryTerm in tfQuery else 0
            documentWeight += docTF * qTF
            
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

In [141]:
def modeloIDF(termos):
    documentos = getDocumentsWithTerms(termos)
    tfQuery = generateQueryTF(termos)
    documentsTF = []
    for idDoc in documentos:
        documentWeight = 0
        for queryTerm in tfQuery:
            docTF = dados.TF[idDoc][queryTerm] if queryTerm in dados.TF[idDoc] else 0
            qTF = tfQuery[queryTerm] if queryTerm in tfQuery else 0
            idf = math.log10(dados.titulo.size/len(indice_invertido[queryTerm]))
            documentWeight += docTF * qTF * idf
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

In [145]:
termos = ["segundo", "turno"]
consultaTF = modeloTF(termos)[:5]
consultaIDF = modeloIDF(termos)[:5]

[2744, 7, 2112, 7672, 2388]
[2744, 2112, 7672, 1235, 2388]

print(consultaTF)
print(consultaIDF)

[2743, 2111, 2387, 2177, 2137]
[(2743, 25.32148212306859), (2111, 14.79657944001845), (2387, 12.772739223519034), (2177, 12.130500211956493), (1963, 10.52490268305014)]
